# Trainers

Designing a training loop for learning a complex dataset.

In [1]:
import os

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
from torchvision import datasets, transforms

from lib.dataset import PCam
from lib.evaluate import evaluate
from lib.models import Veggie16
from lib.train import train
from lib.transforms import ToNormalized, ToClosed

In [2]:
DATASET_PATH = 'C:/Users/Amod/PycharmProjects/CS184a/input'

In [3]:
num_epochs = 2
num_classes = 2
batch_size = 50
learning_rate = 0.001
rgb_means = [178, 137, 176]
rgb_stds = [61, 72, 55]

In [4]:
image_dir = os.path.join(DATASET_PATH, 'train')
csv_path = os.path.join(DATASET_PATH, 'train_labels.csv')
pcam_dataset = PCam(image_dir, csv_path, 
                    transforms.Compose([
                        ToClosed(),
                        transforms.RandomVerticalFlip(),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        ToNormalized(rgb_means, rgb_stds)
                    ]))
print(len(pcam_dataset))

220025


In [5]:
train_set, val_set = utils.data.random_split(pcam_dataset, [154000, 66025])

In [6]:
train_loader = utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=0)
val_loader = utils.data.DataLoader(val_set, batch_size=batch_size, num_workers=0)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda:0


In [8]:
model = Veggie16(2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model)

Veggie16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilat

In [9]:
total_loss = train(model, train_loader, device, criterion, optimizer, num_epochs=num_epochs)

Epoch [1/2], Step [1000/3080], Loss: 0.642245


KeyboardInterrupt: 

In [ ]:
# plt.semilogy()
# plt.plot(tuple(range(num_epochs)), losses, 'b-')
# plt.title('Model Loss')
# plt.xlabel('Epoch')
# plt.ylable('Cross Entropy Loss')
# plt.show()
print(total_loss)

In [ ]:
score, accuracy, loss = evaluate(model, val_loader, device, criterion)
print('Evaluating Veggie16 on Validation set:')
print('F1-Score:', score)
print('Accuracy:', accuracy)
print('Loss:', loss)

In [11]:
torch.save(model.state_dict(), '../models/v16_exp0.pth.tar')